### Входные данные
Форма для сдачи: https://goo.gl/forms/qptFQ1ukuCxeMEA63

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдите минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 3 знака после запятой.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install

#### Генерация данных

In [13]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in tqdm.trange(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [8]:
import numpy as np
import tqdm

win_len = 1000 # window size

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return [round(x, 3) for x in result / count]

In [4]:
#import theano
#from theano import tensor as T
from collections import deque 
import itertools

#vector = T.vector("row")
#mean = T.scalar("mean")
#function_std = theano.function([vector, mean], T.sqrt(T.mean((vector - mean) ** 2)))

def example(stream):
    for value in stream:
        yield (value, value + 10)
#Easy
def calculate_features_easy(stream):
    
    d = deque([], win_len)
    c = 0
    for i in stream:
        if c < win_len:
            d.append(i)
            c += 1
        else:
            break
    
    s = np.sum(d)
    std  = np.std(d)
    median  = np.median(d)
    
    yield ( s / float(win_len),
            std,
            d[0], #! window always consists of sorted elements
            median,
            d[-1] ) #!
    
    c = 0
    for value in stream:
        old = d.popleft()
        d.append(value)
        s = s + ( value - old )
        std  = np.std(d)
        median  = ( d[win_len / 2] + d[ win_len / 2 - 1] ) / 2 #!
        if c % 1000000 == 0:
            print s / float(win_len), std, median
        c += 1
        yield ( s / float(win_len),
                std,
                d[0], #!
                median,
                d[-1] ) #!
get_tuple_stream_mean(calculate_features_easy(easy_stream()), 5)

523 303.835168498 527.0
524.18 303.846753479 528
999898.733 296.244179202 999898
2000038.711 296.027234354 2000039
2999502.424 299.512130345 2999506
3999786.572 282.139463415 3999791
4998940.183 282.74345883 4998947
5998899.229 300.562856253 5998896
6998379.896 278.731489402 6998380
7997275.767 291.900172509 7997281
8998091.797 278.475416852 8998091


[4998569.34, 288.659, 4998069.999, 4998569.173, 4999068.68]

In [14]:
#Medium
def calculate_features_medium(stream):
    
    minimum = 0
    maximum = 0
    max_pos = 0
    min_pos = 0
    
    d = deque([], win_len)
    c = 0
    for i in stream:
        
        if c == 0:
            minimum = i
            maximum = i
            
        if c < win_len:
            d.append(i)
            
            if minimum >= i:
                minimum = i
                min_pos = c
                
            if maximum <= i:
                maximum = i
                max_pos = c
                
            c += 1
        else:
            break
    
    s = np.sum(d)
    std  = np.std(d)
    median  = np.median(d)
    
    yield ( s / float(win_len),
            std,
            minimum, #! window always consists of sorted elements
            median,
            maximum ) #!
    
    c = 0
    for value in stream:
        
        old = d.popleft()
        d.append(value)
        
        min_pos -= 1
        max_pos -= 1
        
        if minimum >= value:
            minimum = value
            min_pos = win_len - 1
        else:
            if min_pos == -1:
                minimum = np.min(d)
            
        if maximum <= value:
            maximum = value
            max_pos = win_len - 1
        else:
            if max_pos == -1:
                maximum = np.max(d)
                
        s = s + ( value - old )
        std  = np.std(d)
        median  = np.median(d) #!
        
        if c % 1000000 == 0:
            print s / float(win_len), std, median
        c += 1
        yield ( s / float(win_len),
                std,
                minimum, #!
                median,
                maximum ) #!
        
get_tuple_stream_mean(calculate_features_medium(medium_stream()), 5)

  0%|          | 1895/10000000 [00:00<18:54, 8809.87it/s] 

127.189 73.5252424613 127.0


 10%|█         | 1001913/10000000 [03:29<31:03, 4829.45it/s]

128.74 72.5471736183 133.0


 20%|██        | 2001807/10000000 [06:57<27:28, 4851.43it/s]

...

126.354 74.7791193048 128.5


 90%|█████████ | 9001828/10000000 [31:08<03:24, 4890.13it/s]

125.999 72.4719324911 123.0


100%|██████████| 10000000/10000000 [34:35<00:00, 4817.34it/s]


[127.494, 73.834, 0.02, 127.522, 254.98]

In [15]:
#Nightmare
def calculate_features_nightmare(stream):
    
    minimum = 0
    maximum = 0
    max_pos = 0
    min_pos = 0
    
    d = deque([], win_len)
    c = 0
    for i in stream:
        
        if c == 0:
            minimum = i
            maximum = i
            
        if c < win_len:
            d.append(i)
            
            if minimum >= i:
                minimum = i
                min_pos = c
                
            if maximum <= i:
                maximum = i
                max_pos = c
                
            c += 1
        else:
            break
    
    s = np.sum(d)
    std  = np.std(d)
    median  = np.median(d)
    
    yield ( s / float(win_len),
            std,
            minimum, #! window always consists of sorted elements
            median,
            maximum ) #!
    
    c = 0
    for value in stream:
        
        old = d.popleft()
        d.append(value)
        
        min_pos -= 1
        max_pos -= 1
        
        if minimum >= value:
            minimum = value
            min_pos = win_len - 1
        else:
            if min_pos == -1:
                minimum = np.min(d)
            
        if maximum <= value:
            maximum = value
            max_pos = win_len - 1
        else:
            if max_pos == -1:
                maximum = np.max(d)
                
        s = s + ( value - old )
        std  = np.std(d)
        median  = np.median(d) #!
        
        if c % 1000000 == 0:
            print s / float(win_len), std, median
        c += 1
        yield ( s / float(win_len),
                std,
                minimum, #!
                median,
                maximum ) #!
        
get_tuple_stream_mean(calculate_features_nightmare(nightmare_stream()), 5)

  0%|          | 1900/10000000 [00:00<18:40, 8919.07it/s] 

496076512.345 290034971.875 506049849.5


 10%|█         | 1001769/10000000 [03:27<31:03, 4827.47it/s]

501892680.033 288686381.928 511772707.0


 20%|██        | 2001602/10000000 [06:54<27:56, 4771.00it/s]

...

514146547.309 293560055.003 541962905.5


 90%|█████████ | 9001927/10000000 [7:47:55<03:35, 4632.92it/s]

497884689.261 290563191.007 500209379.5


100%|██████████| 10000000/10000000 [7:51:26<00:00, 353.53it/s]


[499909697.53, 288455165.982, 552365.285, 499814174.172, 999409864.069]